This is a project about prediting index or stock price in financial market by using machine learning. In this project, we will ulitlze Yahoo Finance API to obtain the data we need. After that, pandas and sklearn library is going to be used in order to build in a machine learning that use to forecast the future price. 

In this project, there are two parts totally. The first part is about defining some necessary code in this project. These defined code is including three main function. 
1. Accessing Yahoo Finance API to obtain all necessary data, then transfer it into a dataframe
2. Ploting data with some useful annotation and common used indicator.
3. Learning data by using LinearRegression function from sklearn.

In the second part, it will show how to use all the function. In this part, I will initally work out some examples of accessing the data by the defined function, explaining how to use it. After that I will try to predict the stock price after it.

# Part 1. Defining Necessary Code

##### Part 1.1 implement all library 

In [2]:
from yahoo_finance import Share
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import style
#from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV

In [3]:
style.use('dark_background')
#%matplotlib osx
#%matplotlib notebook
#%matplotlib inline  
#plt.ioff()
#getting all the styke in matplotlib
#print plt.style.available

##### Part 1.2 Accessing stock and transfer it into dataframe

In [4]:
def get_data(symbols,start_date,end_date):
    ## Getting data from yahoo finance
    dates = pd.date_range(start_date,end_date)
    df = pd.DataFrame(index=dates)
    if '^HSI' not in symbols:
        symbols.insert(0,'^HSI')
    for symbol in symbols:
        df_tmp = Share(symbol).get_historical(start_date,end_date)
        df_tmp = pd.DataFrame(df_tmp)
        df_tmp['Adj_Close'] = df_tmp['Adj_Close'].astype(float)
        df_tmp = df_tmp.rename(columns={'Adj_Close':symbol})
        df_tmp = df_tmp.set_index('Date')
        df_tmp = df_tmp[symbol]
        df = df.join(df_tmp)        
        if symbol == '^HSI':
            df = df.dropna(subset=['^HSI'])
            
        df = df.dropna(subset=['^HSI'])
    df.fillna(method='ffill',inplace=True)
    df.fillna(method='backfill',inplace=True)
    return df.round(2)

##### Part 1.3 Presenting data

In [5]:
def plot_stock(df_data, target_stock, annotate_switch):
    
    if target_stock == 'all':
        df = df_data
        symbols = df_data.columns
        title_name = 'All Stock'
    else:
        df = df_data[target_stock]
        symbols = target_stock
        title_name = 'target_stock'
        
        
    ax = df.plot(title= title_name, fontsize= 10, figsize=(20, 10))
    ax.set_xlabel('Date', fontsize= 15)
    ax.set_ylabel('Price', fontsize= 15)
    if annotate_switch == True:
        bbox_props = dict( boxstyle = 'larrow,pad=0.2', fc= "k", ec= "y",)
        for symbol in symbols:
            ax.annotate( str(df_data[symbol][-1]), ((data.index[-1]), df_data[symbol][-1]),
                        xytext= (data.index[-1]+ dt.timedelta(days=5), df_data[symbol][-1]),
                        color= 'y', fontweight='bold', fontsize = 10, bbox= bbox_props)

    plt.subplots_adjust(left= 0.04, bottom= 0.1, right= 0.95, top= 0.95, wspace= 0, hspace= 0.1 )
    plt.show()
        
        
#         ax = df_data.plot(title= 'All Stock', fontsize= 10)
#         ax.set_xlabel('Date', fontsize= 15)
#         ax.set_ylabel('Price', fontsize= 15)
#         if annotate_switch == True:
#             bbox_props = dict( boxstyle = 'larrow,pad=0.2', fc= "k", ec= "y",)
#             for symbol in df_data.columns:
#                 ax.annotate( str(df_data[symbol][-1]), ((data.index[-1]), df_data[symbol][-1]),
#                             xytext= (data.index[-1]+ dt.timedelta(days=5), df_data[symbol][-1]),
#                             color= 'y', fontweight='bold', fontsize = 10, bbox= bbox_props)
    
#         plt.subplots_adjust(left= 0.04, bottom= 0.1, right= 0.95, top= 0.95, wspace= 0, hspace= 0.1 )
#         plt.show()
        
        
#     else:
#         ax = df_data[target_stock].plot(title= target_stock, fontsize= 10)
#         ax.set_xlabel('Date', fontsize= 15)
#         ax.set_ylabel('Price', fontsize= 15)
#         if annotate_switch == True:
#             bbox_props = dict( boxstyle = 'larrow,pad=0.2', fc= "k", ec= "y",)
#             for symbol in target_stock:
#                 ax.annotate( str(df_data[symbol][-1]), ((data.index[-1]), df_data[symbol][-1]),
#                              xytext= (data.index[-1]+ dt.timedelta(days=5), df_data[symbol][-1]),
#                             color= 'y', fontweight='bold', fontsize = 10, bbox= bbox_props)

#         plt.subplots_adjust(left= 0.04, bottom= 0.1, right= 0.95, top= 0.95, wspace= 0, hspace= 0.1 )
#         plt.show()
    

In [6]:
def plot_datasubplot(df):
    ax = df.plot(subplots= True, figsize= (20,15), title= str(df.columns), secondary_y= False)
    plt.xlabel("Date")
    plt.subplots_adjust(left= 0.03, bottom= 0.1, right= 0.98, top= 0.95, wspace= 0, hspace= 0.1 )
    plt.show()

In [7]:
def log_correlation_finder(dataframe,target,critical_value):
    log_data_dataframe = dataframe.corr()
    high_corr = []
    for index in log_data_dataframe.index:
        if index != target:
            if abs(log_data_dataframe.loc[target][index]) > critical_value:
                high_corr.append(index)
    return log_data_dataframe, high_corr

In [8]:
def ema(stock,window):
    dates = pd.date_range(str(stock.index[0]),str(stock.index[-1]))
#    dates = pd.date_range('2015-07-01','2016-07-31')
    df = pd.DataFrame(index=dates)
    name = '{}_ema({})'.format(stock.name,window)
    df[name] = pd.stats.moments.ewma(stock,window)
#    df = df.join(df[name])
    df = df.dropna(subset=[name])
    return df

In [9]:
def rsi(stock,window):
    dates = pd.date_range(str(stock.index[0]),str(stock.index[-1]))
    df = pd.DataFrame(index=dates)
    name = '{}_RSI({})'.format(stock.name,window)
    df[name] = stock
    df = df.dropna(subset=[name])

    delta = df[name].diff().dropna()
    
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]     
    d[delta < 0] = -delta[delta < 0]
    u[u.index[window-1]] = np.mean( u[:window] ) #first value is sum of avg gains
    u = u.drop(u.index[:(window-1)])
    d[d.index[window-1]] = np.mean( d[:window] ) #first value is sum of avg losses
    d = d.drop(d.index[:(window-1)])
    rs = pd.stats.moments.ewma(u, com=window-1, adjust=False)/pd.stats.moments.ewma(d, com=window-1, adjust=False)
    df = 100 - 100 / (1 + rs)
#    df = pd.DataFrame(df)
    return df

In [10]:
def graph_with_indicator(stock):
    #5,8,13
    df = pd.DataFrame(stock)
    name = str(df.columns[0])
    windows = [5,8,13]
    for window in windows:
        df_tmp = ema(stock,window)
        df = df.join(df_tmp)
        
    stock_rsi = rsi(stock,14) 
    
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20, 10))

    fig.suptitle(name, fontsize=20)
    df.plot(ax=axes[0])
       
    axes[0].xaxis.set_visible(False)
    axes[0].legend(loc='upper center', ncol=4, fancybox=False, shadow=True)
    
    bbox_props = dict( boxstyle = 'square,pad=0.2', fc= "k", ec= "y",)
    axes[0].annotate( str(df.iloc[-1]), ((stock.index[-1]), stock[-1]),
                        xytext= (df.index[-1]+ dt.timedelta(days=10), ((stock.max()+np.percentile(stock,75))/2)),
                        color= 'y', fontweight='bold', fontsize = 12, bbox= bbox_props)
    

    
    
    stock_rsi.plot(ax=axes[1])
    axes[1].xaxis.set_visible(True)
    
    annotates = [stock,stock_rsi]
    bbox_props2 = dict( boxstyle = 'larrow,pad=0.2', fc= "k", ec= "y",)
    for i in range(2):
        axes[i].annotate( str(annotates[i][-1]), ((annotates[i].index[-1]), annotates[i][-1]),
                    xytext= (annotates[i].index[-1]+ dt.timedelta(days=5), annotates[i][-1]),
                    color= 'y', fontweight='bold', fontsize = 10, bbox= bbox_props2)
    
    axes[1].set_ylim([0,100])
    axes[1].plot([stock_rsi.index[-1], stock_rsi.index[0]], [80, 80],linestyle='--',color = '0.3')
    axes[1].plot([stock_rsi.index[-1], stock_rsi.index[0]], [20, 20],linestyle='--',color = '0.3')
    axes[1].legend(loc='upper left')
    plt.subplots_adjust(left= 0.05, bottom= 0.1, right= 0.75, top= 0.9, wspace= 0, hspace= 0.1 )
    plt.show()
    
#     df = df.join(stock_rsi)
#     df = df.iloc[14:,:]
    return df

##### Part 1.4  Machine Learning

In [11]:
def split_data(df):
    data.shape[1]
    splite_rate = 0.3 
    splite = splite_rate * data.shape[0]    
    days = (df.index -  df.index[0]).days.reshape(-1, 1)
    df['days'] = pd.DataFrame(days, index= df.index)
    
    y_train, y_test = df.iloc[:int(splite) ,0:1], df.iloc[int(splite): ,0:1]
    X_train, X_test = df.iloc[:int(splite) ,1:], df.iloc[int(splite): ,1:]
    Y = df.iloc[:,0:1]
    return X_train, X_test, y_train, y_test

In [12]:
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

In [13]:
def fit_model(X, y):
    """ Tunes a decision tree regressor model using GridSearchCV on the input data X 
        and target labels y and returns this optimal model. """

    regressor = linear_model.LinearRegression()

    # Set up the parameters we wish to tune
    parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}

    # Make an appropriate scoring function
    scoring_function = make_scorer(performance_metric)

    # Make the GridSearchCV object
    reg = GridSearchCV(regressor, parameters, scoring=scoring_function)

    # Fit the learner to the data to obtain the optimal model with tuned parameters
    reg.fit(X, y)

    # Return the optimal model
    return reg.best_estimator_

In [14]:
def graph_learing(X_train, X_test, y_train, y_test):
    train_value = reg.predict(X_train)
    test_value = reg.predict(X_test)
    Y = y_test.append(y_train)

    train_value = pd.DataFrame(train_value,index = y_train.index, columns=['train_value'])
    test_value = pd.DataFrame(test_value,index = y_test.index, columns=['test_value'])

    predict = test_value.append(train_value)
    predict = Y.join(predict)
    R_2 = performance_metric(y_test,test_value)
    
    predict['error_train_upper'] = train_value * 1.05
    predict['error_train_lower'] = train_value * 0.95
    predict['error_test_upper'] = test_value * 1.05
    predict['error_test_lower'] = test_value * 0.95
    
    predict['error_test'] = abs(predict['test_value'] - predict[predict.columns[0]])/predict[predict.columns[0]]
    predict['error_train'] = abs(predict['train_value'] - predict[predict.columns[0]])/predict[predict.columns[0]]
    
    
    ax = predict.iloc[:,0:3].plot(title= predict.columns[0], fontsize= 10, figsize=(20, 10))
    ax.set_xlabel('Date', fontsize= 15)
    ax.set_ylabel('Price', fontsize= 15)
    
    plt.fill_between(predict.index, predict['error_train_upper'], predict['error_train_lower'], alpha= 0.3 ,color= 'grey')
    plt.fill_between(predict.index, predict['error_test_upper'], predict['error_test_lower'], alpha= 0.3 ,color= 'grey')
    
    
    ax.text(1, 1, 'Average train error {:.2f} % , Average test error {:.2f} %'
            .format(predict['error_train'].mean() *100, predict['error_test'].mean() *100),
        verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes,
        color='Yellow', fontsize=20)
    plt.subplots_adjust(left= 0.04, bottom= 0.1, right= 0.95, top= 0.95, wspace= 0, hspace= 0.1 )
    print "R Square = ", R_2
    plt.show()    
    return predict

# Part 2. Example of preding Hang Seng Index

##### Part 2.1  Data accessing

get_data is a defined function that obtain the daily stock price we need in a particular time period. In this function, we need three input. 
1. the list of stock we want
2. starting date of the data we need  
3. ending date of the data we need 

Finally, we will have a dataframe that data will be the index and stock price will be the columns of this dataframe.


In this example, I try to obtain the 5 stock price data from Hong Kong, which are Hang Seng China enterprises Index(^HSCE), HSBC HOLDINGS(0005.hk), AIA(1299.hk) and China Mobile(0941.hk). Also the preiod I choose is from 2015-07-01 to 2016-07-31. After that, I will save the dataframe named 'data'

In [20]:
stock = ['^HSCE','0005.hk','1299.hk','0941.hk']
data =  get_data(stock,'2015-07-01','2016-07-31')

In [21]:
data.head()

,^HSI,^HSCE,0005.hk,1299.hk,0941.hk
2015-07-02,26282.32,12784.65,65.92,50.74,97.32
2015-07-03,26064.11,12608.98,65.92,50.00,96.98
2015-07-06,25236.28,12231.43,64.36,48.53,93.54
2015-07-07,24975.31,11827.30,64.88,49.07,92.91
2015-07-08,23516.56,11107.30,62.32,46.18,87.28


In [22]:
data.corr()

,^HSI,^HSCE,0005.hk,1299.hk,0941.hk
^HSI,1.000000,0.977592,0.876642,0.796359,0.845948
^HSCE,0.977592,1.000000,0.916680,0.679833,0.785749
0005.hk,0.876642,0.916680,1.000000,0.551477,0.682594
1299.hk,0.796359,0.679833,0.551477,1.000000,0.687009
0941.hk,0.845948,0.785749,0.682594,0.687009,1.000000


plot_stock  is a defined function that plot the daily stock price we have. In this function, we need three input.
1. The dataframe we have
2. The stock we want to present in the dataframe, if inputing 'all' in will present all stock price in the dataframe, if we just want to present few or one particular stocks we can just input the list of stock name we have.
3. Annotation switch. If input True, there will be a annotation of the last price, versa vice

Here will be two examples, 
1. I will try to plot all the stock price in the dataframe without anntation. However, it is not a good example becasue the price of scale are too different in this dataframe.
2. I will present 0700.hk and 0005.hk with annotation.


In [24]:
plot_datasubplot(data)

In [20]:
plot_stock(data,'all',False)

In [21]:
plot_stock(data,['0700.hk','0005.hk'],True)

plot_datasubplot is ulitized to present all the stock price in subplots. in this example, I am going to present the whole data

graph_with_indicator is a function that present a chosen stock with two common used finance indicator, Exponential Moving Average(EMA) and Relative Strength Index(RSI). 
Three different windows, 5,8,13 , will be utilized for EMA. Moroover, RSI will have window = 14.
Here are the websites of how these indicator to be useful in stock trading. 

[Exponential Moving Average(EMA)][1]
[1]:http://www.investopedia.com/terms/e/ema.asp

[Relative Strength Index(RSI)][2]
[2]:http://www.investopedia.com/terms/r/rsi.asp

Finally, the function will return the dataframe, including the stock price and the EMA data.

In the following example, i will try to present Hang Seng index in graph and create the new dataframe for it

In [25]:
HSI =  graph_with_indicator(data['^HSI'])
HSI.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=True,com=5).mean()
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=True,com=8).mean()
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=True,com=13).mean()
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(ignore_na=False,min_periods=0,adjust=False,com=13).mean()


,^HSI,^HSI_ema(5),^HSI_ema(8),^HSI_ema(13)
2015-07-02,26282.32,26282.320000,26282.320000,26282.320000
2015-07-03,26064.11,26163.296364,26166.797059,26169.174074
2015-07-06,25236.28,25796.564615,25819.461198,25834.901243
2015-07-07,24975.31,25532.196513,25569.811615,25595.558943
2015-07-08,23516.56,24970.539897,25057.221384,25115.967615


log_correlation_finder is a function to produce the correlation table of the dataframe and to find out a certain level of correlation in absolute value bewtween the chosen stock to other.

There is an example to show how to obtain the correclation table and find out all the stock have correction which is higher than 0.8 with 0005.hk (HSBC)

In [83]:
log_data_corr, high_corr_asset = log_correlation_finder(data,'0005.hk',0.8)
print log_data_corr, '\n', high_corr_asset

             ^HSI     ^HSCE   0700.hk   0005.hk   1299.hk   0941.hk
^HSI     1.000000  0.977592 -0.030809  0.876633  0.796233  0.845834
^HSCE    0.977592  1.000000 -0.199795  0.916687  0.679615  0.785587
0700.hk -0.030809 -0.199795  1.000000 -0.422210  0.476924  0.077976
0005.hk  0.876633  0.916687 -0.422210  1.000000  0.551244  0.682382
1299.hk  0.796233  0.679615  0.476924  0.551244  1.000000  0.687239
0941.hk  0.845834  0.785587  0.077976  0.682382  0.687239  1.000000 
['^HSI', '^HSCE']


##### Part 2.2  Machine Learning

split is defined to split data into four dataset, X_train, X_test, y_train, y_test, which has 0.3 splitting rate.

**Note that the first column of the dataframe will the y value, it means that the stock we want to predict must be placing in the first column.**             

fit_model is using LinearRegression from sklearn to learn the data and using R square to be our performance metric , Finally it wll return the best estimator.

in the following example, we will try to predict Hang Seng Index in our dataframe by using these function



In [29]:
X_train, X_test, y_train, y_test= split_data(data)
reg = fit_model(X_train, y_train)
print reg.get_params()

{'copy_X': True, 'normalize': False, 'n_jobs': 1, 'fit_intercept': True}


In [34]:
train_value = reg.predict(X_train)
train_value = pd.DataFrame(train_value,index = y_train.index, columns=['train_value'])
R = performance_metric(y_train,train_value)

print 'R square of training data = ' + str(R)

R square of training data = 0.997933120788


graph_learing is a function to show the predication result in graph and dataframe. The function will firstly predict the data by using training set, then predicting it by using testing set.

After that, the function will plot the true value of the stock price and the predicting value in both training and test set. Moreover, there will be a grey shadow area which is +- 0.5 % of predicting value, it helps us to know how relaiable of this model. 

Furthermore, the average error between predicting value and true value will be label at the top-right of the graph.
Finally, the function will also return the dataframe of the stock price and the predicting value

In [35]:
result = graph_learing(X_train, X_test, y_train, y_test)
result.head()

R Square =  0.903454754554


,^HSI,test_value,train_value,error_train_upper,error_train_lower,error_test_upper,error_test_lower,error_test,error_train
2015-10-28,22956.57,23013.790381,NaN,NaN,NaN,24164.479900,21863.100862,0.002493,NaN
2015-10-29,22819.94,22900.400655,NaN,NaN,NaN,24045.420688,21755.380623,0.003526,NaN
2015-10-30,22640.04,22636.627083,NaN,NaN,NaN,23768.458438,21504.795729,0.000151,NaN
2015-11-02,22370.04,22447.039317,NaN,NaN,NaN,23569.391283,21324.687351,0.003442,NaN
2015-11-03,22568.43,22625.588979,NaN,NaN,NaN,23756.868428,21494.309530,0.002533,NaN


# Part 3. Conclusion

In this project, Hong Kong stock market is being used as an exmaple to show how his program predicting stock price by using machine learning knowledge. I have tried to predict Hang Keng Index with different major stock in Hong Kong. It finally shows that it is a reliable estimator as it has a very high R square and the average error is less then 5%.